In [26]:
import random
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
import csv
import numpy as np
from tabulate import tabulate
from classification import classification
from rule_gen import rule_generation
from daehan_mlutil import utilities
from util import util

with open('../data/dataset.csv', 'r') as file:
    dataset = [list(filter(None, row)) for row in csv.reader(file)]

min_support = 0.1
transactions_df = util.convert_trans_to_df(dataset)
transactions_df

,0,1,ACETAMINOPHEN,ALBUMIN HUMAN 25%,ALBUMIN HUMAN 5%,ALBUTEROL 0.083%,ALBUTEROL 90 MCG,ALBUTEROL CONCENTRATED 0.5%,ALLOPURINOL,ALPRAZOLAM,...,TACROLIMUS,TAMSULOSIN,THIAMINE,TRAMADOL,TRAZODONE,VANCOMYCIN,VANCOMYCIN 1 G/200 ML D5W,VANCOMYCIN 1.25 G/250 ML NS,VANCOMYCIN 1.5 G/500 ML NS,WARFARIN
0,True,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1,True,False,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,True,False,True,False,False,False,True,True,False,False,...,False,False,False,True,True,False,True,True,True,False
4611,True,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,True,False,False,False,False
4612,True,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False
4613,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
random.seed(0)
transactions_0 = pd.DataFrame(
    transactions_df[transactions_df['0']].reset_index(drop=True))
transactions_1 = pd.DataFrame(
    transactions_df[transactions_df['1']].reset_index(drop=True))

In [28]:
indices = list(range(0, len(transactions_0)))
random.shuffle(indices)
test_set_0 = transactions_0.iloc[indices[:417], :].reset_index(drop=True)
training_set_0 = transactions_0.iloc[indices[417:], :].reset_index(drop=True)

indices = list(range(0, len(transactions_1)))
random.shuffle(indices)
test_set_1 = transactions_1.iloc[indices[:43], :].reset_index(drop=True)
training_set_1 = transactions_1.iloc[indices[43:], :].reset_index(drop=True)

training_set = pd.concat([training_set_0, training_set_1])
test_set = pd.concat([test_set_0, test_set_1])
training_set

,0,1,ACETAMINOPHEN,ALBUMIN HUMAN 25%,ALBUMIN HUMAN 5%,ALBUTEROL 0.083%,ALBUTEROL 90 MCG,ALBUTEROL CONCENTRATED 0.5%,ALLOPURINOL,ALPRAZOLAM,...,TACROLIMUS,TAMSULOSIN,THIAMINE,TRAMADOL,TRAZODONE,VANCOMYCIN,VANCOMYCIN 1 G/200 ML D5W,VANCOMYCIN 1.25 G/250 ML NS,VANCOMYCIN 1.5 G/500 ML NS,WARFARIN
0,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,True,True,False,False
1,True,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,True,False,True,False,False,False,True,True,False,False,...,False,True,False,False,False,False,True,True,False,False
3,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,False,True,True,False,False,False,True,True,False,False,...,False,False,False,False,False,True,False,False,False,False
389,False,True,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,True,True,False,False
390,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
391,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
test_set

,0,1,ACETAMINOPHEN,ALBUMIN HUMAN 25%,ALBUMIN HUMAN 5%,ALBUTEROL 0.083%,ALBUTEROL 90 MCG,ALBUTEROL CONCENTRATED 0.5%,ALLOPURINOL,ALPRAZOLAM,...,TACROLIMUS,TAMSULOSIN,THIAMINE,TRAMADOL,TRAZODONE,VANCOMYCIN,VANCOMYCIN 1 G/200 ML D5W,VANCOMYCIN 1.25 G/250 ML NS,VANCOMYCIN 1.5 G/500 ML NS,WARFARIN
0,True,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,True,False,False,False,True,True,False,False,...,False,False,False,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
39,False,True,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,True,False,False
40,False,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41,False,True,True,True,False,False,False,True,False,False,...,False,False,True,False,False,False,True,False,False,False


In [30]:
import multiprocessing
import pandas as pd
from posneg_rule_gen.posneg_rule_generation import ponerg
from rule_gen import apriori_mlx
from util import util
from daehan_mlutil import utilities

def ponerg_parallel(item):
    return ponerg(item, classes, class_support_count_dict, transactions_df)

rules = []
transactions_df = training_set
X_df = pd.DataFrame(transactions_df.drop(['1', '0'], axis=1))
classes = [frozenset(['0']), frozenset(['1'])]
class_support_count_dict = util.get_support_count_dict_df(classes, transactions_df)

f1, previous_itemset_arr = apriori_mlx.apriori_of_size_1(X_df, min_support=min_support)
f1 = f1.tolist()
freq_itemsets = [f1]
for item in f1:
    rules.extend(ponerg(item, classes, class_support_count_dict, transactions_df))
np.expand_dims(np.array(np.array(f1)), axis=1)

array([[frozenset({'ACETAMINOPHEN'})],
       [frozenset({'ALBUTEROL 90 MCG'})],
       [frozenset({'ALBUTEROL CONCENTRATED 0.5%'})],
       [frozenset({'AMLODIPINE'})],
       [frozenset({'ASPIRIN'})],
       [frozenset({'ASPIRIN EC'})],
       [frozenset({"ASPIRIN, CHILDREN'S"})],
       [frozenset({'ATORVASTATIN'})],
       [frozenset({'BISACODYL'})],
       [frozenset({'CALCIUM GLUCONATE 2 G/50 ML NS'})],
       [frozenset({'CEFAZOLIN'})],
       [frozenset({'CEFAZOLIN 2 G/100 ML NS'})],
       [frozenset({'CEFEPIME'})],
       [frozenset({'CEFTRIAXONE'})],
       [frozenset({'CHLORHEXIDINE GLUCONATE 0.12%'})],
       [frozenset({'DEXTROSE 5%-NACL 0.9%'})],
       [frozenset({'DEXTROSE 50%-WATER'})],
       [frozenset({'DIAZEPAM'})],
       [frozenset({'DIPHENHYDRAMINE HCL'})],
       [frozenset({'DOCUSATE SODIUM'})],
       [frozenset({'ENOXAPARIN'})],
       [frozenset({'FAMOTIDINE'})],
       [frozenset({'FENTANYL 2000 MCG/100 ML NS'})],
       [frozenset({'FENTANYL CITRATE (PF)

In [31]:
import time

start_time = time.time()
k = 0
while freq_itemsets[k] is not None and len(freq_itemsets[k]) > 0:
    k_freq_itemsets, previous_itemset_arr = apriori_mlx.apriori_of_size_k(
        X_df, previous_itemset_arr, min_support=min_support, k=k + 2, low_memory=True)
    if not k_freq_itemsets.empty:
        k_freq_itemsets = k_freq_itemsets.tolist()
        # for item in k_freq_itemsets_x_c:
        #     rules.extend(ponerg(item, c, len(transactions_per_c), transactions_df))
        with multiprocessing.Pool() as pool:
            result = pool.map(ponerg_parallel, k_freq_itemsets)
        rules_to_extend = [x[0] for x in result if x != []]
        rules.extend(rules_to_extend)
        freq_itemsets.append(k_freq_itemsets)
    else:
        freq_itemsets.append(None)
    k += 1
time_sec = time.time() - start_time
time_min = time_sec / 60
print("\nProcessing time of %s(): %.2f seconds (%.2f minutes)."
      % ("rule_gen", time.time() - start_time, time_min))
np.expand_dims(np.array(np.array(rules)), axis=1)


Processing time of rule_gen(): 3.56 seconds (0.06 minutes).


array([[{'antecedent': frozenset({'ACETAMINOPHEN'}), 'consequent': '0', 'confidence': 1.024906739112805}],
       [{'antecedent': frozenset({'ALBUTEROL 90 MCG'}), 'consequent': '1', 'confidence': 1.1606304693259821}],
       [{'antecedent': frozenset({'ALBUTEROL CONCENTRATED 0.5%'}), 'consequent': '1', 'confidence': 1.2234574485816194}],
       ...,
       [{'antecedent': frozenset({'MAGNESIUM SULF 2 G/50 ML NS', 'ACETAMINOPHEN', 'DEXTROSE 50%-WATER', 'HEPARIN 5,000 UNIT/ML', 'GLUCAGON', 'GLUCOSE', 'ONDANSETRON', 'FENTANYL CITRATE (PF)'}), 'consequent': '0', 'confidence': 1.0649263645118439}],
       [{'antecedent': frozenset({'OXYCODONE IR', 'MAGNESIUM SULF 2 G/50 ML NS', 'ACETAMINOPHEN', 'MIDAZOLAM', 'DEXTROSE 50%-WATER', 'HEPARIN 5,000 UNIT/ML', 'ONDANSETRON', 'FENTANYL CITRATE (PF)'}), 'consequent': '0', 'confidence': 1.070198277207447}],
       [{'antecedent': frozenset({'OXYCODONE IR', 'MAGNESIUM SULF 2 G/50 ML NS', 'ACETAMINOPHEN', 'DEXTROSE 50%-WATER', 'HEPARIN 5,000 UNIT/ML', 

In [37]:
rules_0 = [rule for rule in rules if rule['consequent'] == '0']
rules_1 = [rule for rule in rules if rule['consequent'] == '1']
sorted_rules = sorted(rules, key=lambda d: d['confidence'], reverse=True)

training_transactions_1 = training_set[training_set['1']].drop(['1', '0'], axis=1).apply(
    lambda row: frozenset(row.index[row]), axis=1).tolist()
scores_training = [classification.predict_proba(object_o, sorted_rules) for object_o in training_transactions_1]
np.array(scores_training)

array([3.99631104, 3.6985775 , 3.92407919, 3.99631104, 3.90561293,
       2.88437833, 3.92407919, 3.42956436, 3.87587258, 3.99631104,
       3.92407919, 3.99631104, 3.90561293, 3.92407919, 3.99631104,
       3.99631104, 3.99631104, 3.04476309, 3.99631104, 1.83503914,
       3.99631104, 3.87587258, 3.90561293, 3.73647261, 3.42956436,
       3.92407919, 3.92407919, 3.92407919, 3.99631104, 3.99631104,
       3.99631104, 3.99631104, 3.99631104, 3.8315137 , 3.6985775 ,
       3.99631104, 3.99631104, 3.99631104, 2.57690101, 3.92407919,
       3.92407919, 2.18913336, 3.50319581, 3.92407919, 3.99631104,
       3.92407919, 3.99631104, 3.99631104, 3.92407919, 3.50319581,
       3.42956436, 3.42956436, 3.90561293, 3.97728099, 2.32310957,
       3.99631104, 3.99631104, 3.99631104, 3.99631104, 3.99631104,
       3.99631104, 3.19340593, 3.99631104, 3.99631104, 3.99631104,
       3.99631104, 3.90561293, 3.90561293, 3.42956436, 2.32310957,
       3.99631104, 3.92407919, 3.42956436, 3.92407919, 3.99631

In [38]:
mean = np.mean(scores_training)
mean

3.713509919397238

In [39]:
test_transactions = test_set.drop(['1', '0'], axis=1).apply(lambda row: frozenset(row.index[row]), axis=1).tolist()
scores_test = [classification.predict_proba(object_o, sorted_rules) for object_o in test_transactions]
scores_test = np.array(scores_test)
scores_test

array([3.06944102, 1.31386645, 3.42956436, 2.41021029, 2.12838459,
       1.81625554, 3.19340593, 2.57690101, 1.8397655 , 3.42956436,
       2.1956473 , 2.08540514, 1.70770113, 3.27360109, 3.27360109,
       3.42956436, 3.90561293, 2.12838459, 3.42956436, 3.42956436,
       3.92407919, 1.64056331, 3.83813054, 3.0332799 , 2.41589237,
       1.33031697, 3.92407919, 1.6843474 , 3.27360109, 1.7242735 ,
       2.30673144, 2.57690101, 3.25642272, 2.30673144, 1.93047703,
       2.30673144, 2.41589237, 1.39971568, 2.18271362, 3.92407919,
       3.99631104, 1.87677854, 3.50319581, 3.92407919, 1.8892474 ,
       1.18068741, 2.30673144, 2.18913336, 3.99631104, 2.05643964,
       2.54701596, 1.59859395, 1.95438549, 3.92407919, 1.83035137,
       3.92407919, 1.07529114, 2.82753417, 1.83490001, 3.50319581,
       3.42956436, 2.03317675, 2.03791118, 1.88969524, 3.92407919,
       2.76147887, 3.92407919, 3.92407919, 1.785581  , 2.82753417,
       2.76147887, 2.27216419, 1.76208651, 1.59035774, 2.30673

In [35]:
y_test = test_set.apply(lambda row: 0 if row['0'] else 1, axis=1).tolist()
y_test = np.array(y_test)
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [36]:
y_pred = np.zeros(len(scores_test), dtype=int)
y_pred[scores_test >= mean] = 1
y_pred

array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,